In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_movie = pd.read_csv('movie.csv')
data_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [3]:
data_rating = pd.read_csv('rating.csv')
data_rating

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [4]:
df = pd.merge(data_movie, data_rating, on = 'movieId')

In [5]:
df.isnull().sum()

movieId      0
title        0
genres       0
userId       0
rating       0
timestamp    0
dtype: int64

In [ ]:
df.drop('timestamp', axis = 1, inplace=True)

In [9]:
merged_data=df
df[:10]

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,4.0
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.5
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,16,3.0
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,5.0


In [10]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


# Input the userId
user_id = int(input("Enter the userId: "))

# Filter data for the selected user
user_data = merged_data[merged_data['userId'] == user_id]

# Select 5 random movies from the user's data
random_movies = user_data.sample(n=5)

# Extract genres from the selected random movies (assuming genres are pipe-separated)
random_movies['genre_list'] = random_movies['genres'].str.split('|')

# Print the selected random movies and their genres
#print("Random")
#print(random_movies[['movieId', 'title', 'genres', 'genre_list']])

# Create a one-hot encoded dataframe for genres
genre_dummies = merged_data['genres'].str.get_dummies('|')

# Only include movies with rating > 3 (optional if needed)
high_rated_movies = merged_data[merged_data['rating'] > 3]

# One-hot encoding for genres of movies with rating > 3
genre_dummies_high_rated = high_rated_movies['genres'].str.get_dummies('|')

# Use apriori to find frequent itemsets with minimum support
frequent_itemsets = apriori(genre_dummies_high_rated, min_support=0.1, use_colnames=True)

# Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Inspect the top rules based on lift
print(rules[['antecedents', 'consequents', 'lift']].head())

# Now, based on the genres of the 5 random movies, we can recommend similar movies
# Extract the genres from the random movies
random_genres = random_movies['genre_list'].explode().unique()

# Recommend movies that match the genres of the random movies
recommended_movies = high_rated_movies[high_rated_movies['genres'].str.contains('|'.join(random_genres))]

# Get distinct recommended movie titles (with rating > 3)
recommended_movie_titles = recommended_movies[['movieId','title']].drop_duplicates()

# Print the recommended movie titles
print("Recommended Movies:")
print(recommended_movie_titles)


Enter the userId: 5249


/Users/spoorthypabba/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


   antecedents  consequents      lift
0  (Adventure)     (Action)  2.051337
1     (Action)  (Adventure)  2.051337
2   (Thriller)     (Action)  1.711771
3     (Action)   (Thriller)  1.711771
4    (Romance)     (Comedy)  1.575590
Recommended Movies:
          movieId                                              title
0               1                                   Toy Story (1995)
49695           2                                     Jumanji (1995)
71938           3                            Grumpier Old Men (1995)
84681           4                           Waiting to Exhale (1995)
87435           5                 Father of the Bride Part II (1995)
...           ...                                                ...
20000256   131250                              No More School (2000)
20000257   131252  Forklift Driver Klaus: The First Day on the Jo...
20000258   131254                       Kein Bund für's Leben (2007)
20000259   131256                      Feuer, Eis & Dosenbier 

In [11]:
# Get the list of movies the user has already watched
watched_movies = user_data[['movieId', 'title']]

# Print the list of movies that the user has already watched
print("\nMovies Already Watched by User:")
print(watched_movies)

# Now, we filter out the movies that the user has already watched from the recommended movies
# This will give us movies that the user has not yet watched
new_recommended_movies = recommended_movie_titles[~recommended_movie_titles['movieId'].isin(watched_movies['movieId'])]

# Print the new recommended movies (not watched by the user)
print("\nNew Recommended Movies (Not Watched by User):")
print(new_recommended_movies)


Movies Already Watched by User:
          movieId                                              title
247345         19              Ace Ventura: When Nature Calls (1995)
1775397       266                         Legends of the Fall (1994)
2409928       350                                 Client, The (1994)
2648137       368                                    Maverick (1994)
3006693       442                              Demolition Man (1993)
3384475       508                                Philadelphia (1993)
3403869       509                                  Piano, The (1993)
4076554       594             Snow White and the Seven Dwarfs (1937)
4412326       708                Truth About Cats & Dogs, The (1996)
4574536       750  Dr. Strangelove or: How I Learned to Stop Worr...
4733160       786                                      Eraser (1996)
6375051      1208                              Apocalypse Now (1979)
8014760      1517  Austin Powers: International Man of Mystery (1...
8